<a href="https://colab.research.google.com/github/ParticleEM/ParEM_anonymous/blob/main/bayesian_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description:** This notebook demonstrates the application of PGA, PQN, PMGA, and SOUL to the Bayesian logistic regression example in Section 4.1 of 'Scalable particle-based alternatives to EM' and reproduces Figure 3 and Table 2 therein. 

# Figure 1

First, we load the modules that we will need:

In [ ]:
#@title Load modules.

# Install the wget package on Colab (if running the notebook locally,
# comment the following line out).
!pip install wget

# OS and wget to load dataset.
import os
import wget

# Numpy for computations.
import numpy as np

# Pyplot for plots.
import matplotlib.pyplot as plt 

Next, we load and curate the dataset:

In [ ]:
#@title Load and normalize the Wisconsin Breast Cancer dataset.

# Fetch dataset from repository:
wget.download('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data')

# Load dataset:
dataset = np.loadtxt('breast-cancer-wisconsin.data', dtype=str, delimiter=',')

# Delete local copy of dataset to avoid duplicates with multiple notebook runs:
os.remove('breast-cancer-wisconsin.data')

# Remove datapoints with missing attributes and change dtype to float:
dataset = dataset[~(dataset == '?').any(axis=1), :].astype(float)

# Extract features and labels, and normalize features:
features = np.array(dataset[:, 1:10] - dataset[:, 1:10].mean(0))
features = features/features.std(0)
labels = np.array([(dataset[:, 10]-2)/2]).transpose()

We then implement the algorithms. They take the following inputs:

*   l : training set labels,
*   f : training set D-dimensional feature vectors,
*   h : step-size,
*   K : number of steps,
*   N : number of particles,
*   th : 1-dimensional vector with parameter guess,
*   X : D x N matrix storing the initial particle cloud.

They return the following outputs:

*   th : K-dimensional vector of parameter estimates,
*   X : D x (KN) matrix storing the particle clouds (the kth cloud is X[:, (k-1)$^\ast$N : k$^\ast$N]).

In [ ]:
#@title Implement algorithms.


# Algorithms.

def pga(l, f, h, K, N, th, X):
    D = f[0, :].size  # Extract latent variable dimension.
    for k in range(K):
        Xk = X[:, -N:]  # Extract current particle cloud.
        #Update particle cloud:
        Xkp1 = (Xk + h*grad_x(th[k], Xk, l, f)
                   + np.sqrt(2*h)*np.random.normal(0, 1, (D, N)))
        X = np.append(X, Xkp1, axis=1) # Store updated cloud.
        th = np.append(th, th[k] + h*ave_grad_th(th[k], Xk))  # Update theta.
    return th, X


def pqn(l, f, h, K, N, th, X):
    D = f[0, :].size  # Extract latent variable dimension.
    for k in range(K):
        Xk = X[:, -N:]  # Extract current particle cloud.
        # Update particle cloud:
        Xkp1 = (Xk + h*grad_x(th[k], Xk, l, f)
                   + np.sqrt(2*h)*np.random.normal(0, 1, (D, N)))
        X = np.append(X, Xkp1, axis=1)  # Store updated cloud.
        # Update theta:
        th = np.append(th, th[k] 
                           + h*(ave_neg_hess_th(D)**-1)*ave_grad_th(th[k], Xk))  
    return th, X


def pmga(l, f, h, K, N, th, X):
    D = f[0, :].size  # Extract latent variable dimension.
    for k in range(K):
        Xk = X[:, -N:]  # Extract current particle cloud.
        # Update particle cloud:
        Xkp1 = (Xk + h*grad_x(th[k], Xk, l, f)
                   + np.sqrt(2*h)*np.random.normal(0, 1, (D, N)))
        X = np.append(X, Xkp1, axis=1)  # Store updated cloud.
        th = np.append(th, theta_opt(Xkp1))  # Update theta.
    return th, X


def soul(l, f, h, K, N, th, X):
    D = f[0, :].size  # Extract latent variable dimension.
    for k in range(K):
        # Run ULA chain:
        for n in range(N):
            Xkn = X[:, -1].reshape(D, 1)  # Extract current particle position.
            # Take a step:
            Xknp1 = (Xkn + h*grad_x(th[k], Xkn, l, f)
                          + np.sqrt(2*h)*np.random.normal(0, 1, (D, 1)))
            X = np.append(X, Xknp1, axis=1)  # Store new particle position.
        th = np.append(th, th[k] + h*ave_grad_th(th[k], X[:, -N:]))  # Update theta.
    return th, X


# Auxiliary functions.

def ave_grad_th(th, x):
    """Returns theta-gradient of log density averaged over particle cloud."""
    return ((x-th).sum(0)).mean()/5


def ave_neg_hess_th(D):
    """Returns negative-theta-Hessian of log density averaged over particles.
    """
    return D/5


def grad_x(th, x, l, f):
    """Returns x-gradient of log density vectorized over particles."""
    s = 1/(1+np.exp(- np.matmul(f, x)))
    return np.matmul((l-s).transpose(), f).transpose() - (x-th)/5


def theta_opt(X):
    return X.mean()  # Return optimal parameter for particle cloud X.

## Figure 3a

We run the algorithms using an 80/20 training/test split of the data:

In [ ]:
# Split data into 80/20 training and testing sets:
from sklearn.model_selection import train_test_split
ftrain, ftest, ltrain, ltest = train_test_split(features, labels, test_size=0.2, 
                                                random_state=0)

# Set approximation parameters:
h = 1e-2 # Step-size. 
K = 400  # Number of steps.
N = 100  # Number of particles.
D = features[0, :].size  # Dimension of latent space.

# Initialize parameter estimates and particle cloud, all at zero:
th0 = np.array([[0]])  # Parameter estimate.
X0 = np.zeros((D, N))  # Particle cloud.

# Run algorithms:
th_pga, X_pga = pga(ltrain, ftrain, h, K, N, th0, X0)
th_pqn, X_pqn = pqn(ltrain, ftrain, h, K, N, th0, X0)
th_pmga, X_pmga = pmga(ltrain, ftrain, h, K, N, th0, X0)
th_soul, X_soul = soul(ltrain, ftrain, h, K, N, th0, X0)

And plot the parameter estimates as a function of k:

In [ ]:
plt.plot(th_pga, label='PGA') 
plt.plot(th_pqn, label='PQN')
plt.plot(th_pmga, label='PMGA')
plt.plot(th_soul, label='SOUL')
plt.xlim([-K/100, K])
plt.legend(loc='upper right')

The estimates produced by all four algorithms converge to the same limit after similar transients. To observe more significant differences between the algorithms' outputs, we examine the posterior approximations they return...

## Figure 3b

The posterior approximations produced by SOUL are more peaked than those produced by PGA, PQN, and PMGA:

In [ ]:
# Extract final particle clouds X^{1:N}_K:
q_pga = X_pga[:, -N:]
q_pqn = X_pqn[:, -N:]
q_pmga = X_pmga[:, -N:]
q_soul = X_soul[:, -N:]

# Generate and plot KDEs:
from scipy import stats  # stats to generate KDEs.
fig = plt.figure(figsize=(27,3), dpi= 100)
for i in range(D):
    # Generate KDEs for ith entry of the final particle cloud X^{1:N}_K:
    kde_min = np.min([q_pga[i, :], q_pqn[i, :], q_pmga[i, :], q_soul[i, :]])
    kde_max = np.max([q_pga[i, :], q_pqn[i, :], q_pmga[i, :], q_soul[i, :]])
    xaxis = np.linspace(kde_min, kde_max, num=100)

    kde_pga = stats.gaussian_kde(q_pga[i, :])(xaxis)
    kde_pqn = stats.gaussian_kde(q_pqn[i, :])(xaxis)
    kde_pmga = stats.gaussian_kde(q_pmga[i, :])(xaxis)
    kde_soul = stats.gaussian_kde(q_soul[i, :])(xaxis)

    # Plot KDEs:
    plt.subplot(1, D, i+1)
    plt.plot(xaxis, kde_pga, label='PGA')
    plt.plot(xaxis, kde_pqn, label='PQN')
    plt.plot(xaxis, kde_pmga, label='PMGA')
    plt.plot(xaxis, kde_soul, label='SOUL')
    plt.title('Feature ' + str(i+1))
    plt.ylim([0, 1.02*np.max([kde_pga, kde_pqn, kde_pmga, kde_soul])])
    plt.xlim([kde_min, kde_max])
    
    
handles, figlabels = plt.gca().get_legend_handles_labels()
fig.legend(handles, figlabels, ncol=4, bbox_to_anchor=(0.36,1.3), 
           loc="upper center",fontsize=20)
plt.subplots_adjust(hspace=0.6)


## Figure 3c

To spot clear differences between behaviour of the SOUL parameter estimates and that of the PGA, PQN, and PMGA parameter estimates, we initialize the estimates far from the optimal parameter and we initialize the particle cloud far from the corresponding posterior mode:

In [ ]:
th0 = np.array([[10]])
X0 = 10*np.ones((D, N))

With this initialization, the PGA, PQN, and PMGA estimates exhibit slower transients than the SOUL estimates:

In [ ]:
# Re-run algorithms:
th_pga, X_pga = pga(ltrain, ftrain, h, K, N, th0, X0)
th_pqn, X_pqn = pqn(ltrain, ftrain, h, K, N, th0, X0)
th_pmga, X_pmga = pmga(ltrain, ftrain, h, K, N, th0, X0)
th_soul, X_soul = soul(ltrain, ftrain, h, K, N, th0, X0)

# Plot parameter estimates:
plt.plot(th_pga, label='PGA') 
plt.plot(th_pqn, label='PQN')
plt.plot(th_pmga, label='PMGA')
plt.plot(th_soul, label='SOUL')
plt.xlim([-K/100, K])
plt.legend(loc='upper right')

However, we can cheaply burn off these long transients using a single-particle run:

In [ ]:
# Warm-start using a single-particle run:
th_burn, X_burn = pmga(ltrain, ftrain, h, K, 1, th0, X0)
th0 = th_burn[-1].reshape((1, 1))
X0 = X_burn[:, -1].reshape((D, 1))*np.ones((1, N))  

# Run the warm-started algorithms:
th_pga, X_pga = pga(ltrain, ftrain, h, K, N, th0, X0)
th_pqn, X_pqn = pqn(ltrain, ftrain, h, K, N, th0, X0)
th_pmga, X_pmga = pmga(ltrain, ftrain, h, K, N, th0, X0)
th_soul, X_soul = soul(ltrain, ftrain, h, K, N, th0, X0)

The estimates produced by all four warm-started algorithms are similar:

In [ ]:
plt.plot(th_pga, label='PGA') 
plt.plot(th_pqn, label='PQN')
plt.plot(th_pmga, label='PMGA')
plt.plot(th_soul, label='SOUL')
plt.xlim([-K/100, K])
plt.ylim([0.75, 1.25])
plt.legend(loc='upper right')

# Table 2

We benchmark performance of the algorithms by running them several times. First, we examine their predictive performance, which we evaluate using two metrics: the test error and log pointwise predictive density (see Appendix F.2 for definitions). 

In [ ]:
#@title Performance metrics.

def predict(f, X):
    """Returns label maximizing the approximate posterior predictive 
    distribution defined by the cloud X, vectorized over feature vectors f.
    """
    s = 1/(1+np.exp(- np.matmul(f, X))).mean(1)
    out = np.zeros((f[:, 0].size, 1))
    out[s >= 1/2] = 1
    return out


def test_error(f, l, X):
    """Returns fraction of misclassified test points."""
    return (np.abs(l - predict(f, X))).mean()


def lppd(f, l, X):
    """Returns log pointwise predictive density."""
    s = 1/(1+np.exp(- np.matmul(f, X)))
    return np.log((((s ** l) * ((1-s) ** (1-l))).mean(1))).mean()

## Predictive performance

We compute the metrics for M=100 runs.

In [ ]:
M = 100  # Number of runs.
N = 100 # Number of particles.
kb = int(K/2)  # Number of burn-in steps.

# Set initial conditions to zero:
th0 = np.array([[0]])  # Parameter estimates.
X0 = np.zeros((D, N))  # Particle cloud.

# Initialize arrays storing the LPPDs and test errors:
lppd_pga = np.zeros((M))
lppd_pqn = np.zeros((M))
lppd_pmga = np.zeros((M))
lppd_soul = np.zeros((M))

error_pga = np.zeros((M))
error_pqn = np.zeros((M))
error_pmga = np.zeros((M))
error_soul = np.zeros((M))

for m in range(M):
    # Randomize the 80/20 testing/training split of the data:
    ftrain, ftest, ltrain, ltest = train_test_split(features, labels, 
                                                    test_size=0.2, 
                                                    random_state=0)

    # Run the algorithms:
    _, X_pga = pga(ltrain, ftrain, h, K, N, th0, X0)
    _, X_pqn = pqn(ltrain, ftrain, h, K, N, th0, X0)
    _, X_pmga = pmga(ltrain, ftrain, h, K, N, th0, X0)
    _, X_soul = soul(ltrain, ftrain, h, K, N, th0, X0)

    #Compute LPPDs:
    lppd_pga[m] = lppd(ftest, ltest, X_pga[:, N*kb:])
    lppd_pqn[m] = lppd(ftest, ltest, X_pqn[:, N*kb:])
    lppd_pmga[m] = lppd(ftest, ltest, X_pmga[:, N*kb:])
    lppd_soul[m] = lppd(ftest, ltest, X_soul[:, N*kb:])

    # Compute test errors:
    error_pga[m] = test_error(ftest, ltest, X_pga[:, N*kb:])
    error_pqn[m] = test_error(ftest, ltest, X_pqn[:, N*kb:])
    error_pmga[m] = test_error(ftest, ltest, X_pmga[:, N*kb:])
    error_soul[m] = test_error(ftest, ltest, X_soul[:, N*kb:])

#Print statistics (mean +- standard deviation):
print('LPPDs:')
print('PGA: '+str(lppd_pga.mean())+' \u00B1 '+str(lppd_pga.std())+'.')
print('PQN: '+str(lppd_pqn.mean())+' \u00B1 '+str(lppd_pqn.std())+'.')
print('PMGA: '+str(lppd_pmga.mean())+' \u00B1 '+str(lppd_pmga.std())+'.')
print('SOUL: '+str(lppd_soul.mean())+' \u00B1 '+str(lppd_soul.std())+'.')

print('Test errors:')
print('PGA: '+str(error_pga.mean())+' \u00B1 '+str(error_pga.std())+'.')
print('PQN: '+str(error_pqn.mean())+' \u00B1 '+str(error_pqn.std())+'.')
print('PMGA: '+str(error_pmga.mean())+' \u00B1 '+str(error_pmga.std())+'.')
print('SOUL: '+str(error_soul.mean())+' \u00B1 '+str(error_soul.std())+'.')

## Stationary variance

Next, we compute the stationary variances of the parameter estimates. To ensure that all algorithms reach stationarity, we warm-start them using a single particle run:

In [ ]:
# Warm-start using a single-particle run:
th_burn, X_burn = pmga(ltrain, ftrain, h, K, 1, th0, X0)
th0 = th_burn[-1].reshape((1, 1))
X0 = X_burn[:, -1].reshape((D, 1))*np.ones((1, N))  

# Initialize arrays storing the stationary variances:
var_pga = np.zeros((M))
var_pqn = np.zeros((M))
var_pmga = np.zeros((M))
var_soul = np.zeros((M))

for m in range(M):
    # Randomize the 80/20 testing/training split of the data:
    ftrain, ftest, ltrain, ltest = train_test_split(features, labels, 
                                                    test_size=0.2, 
                                                    random_state=0)

    # Run the algorithms:
    th_pga, _ = pga(ltrain, ftrain, h, K, N, th0, X0)
    th_pqn, _ = pqn(ltrain, ftrain, h, K, N, th0, X0)
    th_pmga, _ = pmga(ltrain, ftrain, h, K, N, th0, X0)
    th_soul, _ = soul(ltrain, ftrain, h, K, N, th0, X0)

    # Compute variances:
    var_pga[m] = th_pga[kb:].var()
    var_pqn[m] = th_pqn[kb:].var()
    var_pmga[m] = th_pmga[kb:].var()
    var_soul[m] = th_soul[kb:].var()


# Print statistics (mean +- standard deviation):
print('Stationary variances:')
print('PGA: '+str(var_pga.mean())+' \u00B1 '+str(var_pga.std())+'.')
print('PQN: '+str(var_pqn.mean())+' \u00B1 '+str(var_pqn.std())+'.')
print('PMGA: '+str(var_pmga.mean())+' \u00B1 '+str(var_pmga.std())+'.')
print('SOUL: '+str(var_soul.mean())+' \u00B1 '+str(var_soul.std())+'.')

## Runtime

Lastly, we obtain runtime statistics using Python's timeit module (note that Colab can be set to run on a CPU, GPU, or TPU by selecting Runtime --> Change runtime type --> Hardware accelerator):

In [ ]:
from timeit import repeat

# Time the algorithms:
t_pga = np.array(repeat(stmt='pga(ltrain, ftrain, h, K, N, th0, X0)',
                        number=1, repeat=M, globals=globals()))
t_pqn = np.array(repeat(stmt='pqn(ltrain, ftrain, h, K, N, th0, X0)',
                        number=1, repeat=M, globals=globals()))
t_pmga = np.array(repeat(stmt='pmga(ltrain, ftrain, h, K, N, th0, X0)',
                         number=1, repeat=M, globals=globals()))
t_soul = np.array(repeat(stmt='soul(ltrain, ftrain, h, K, N, th0, X0)',
                         number=1, repeat=M, globals=globals()))

# Print statistics (mean +- standard deviation):
print('Runtimes:')
print('PGA: '+str(t_pga.mean())+' \u00B1 '+str(t_pga.std())+'.')
print('PQN: '+str(t_pqn.mean())+' \u00B1 '+str(t_pqn.std())+'.')
print('PMGA: '+str(t_pmga.mean())+' \u00B1 '+str(t_pmga.std())+'.')
print('SOUL: '+str(t_soul.mean())+' \u00B1 '+str(t_soul.std())+'.')